In [1]:
#import general packages
import numpy as np
from scipy.stats import spearmanr
from colorama import Fore, Style, Back
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import exposures_climada as ex
import impact as cimp
import bound_prot_dam as bpd
import n_fct_t_rl_thm_ll as bond_fct
import prem_ibrd as prib
import haz_int_grd as hig
import set_nominal as snom
import functions as fct
import calc_premium as cp
import simulate_bond as sb
import alt_pay_opt as apo

c:\Users\kaibe\miniforge3\envs\climada_env\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
#define minimum return period to be covered
lower_rp = 0.05
lower_share = 0.01
#define maximum return period to be covered
upper_rp = 250

#define benchmark sharpe ratio
target_sharpe = 0.5

#define the risk free rate
rf_rate = 0.05

In [3]:
bond_metrics_gre, returns_gre, premium_dic_gre, nominal_gre, pay_dam_df_gre, es_metrics_gre, int_grid_gre, imp_per_event_flt_gre, imp_admin_evt_flt_gre = bond_fct.sng_cty_bond(country=308,
                                                                                                                                                                                prot_share=upper_rp, 
                                                                                                                                                                                rf_rate=rf_rate, 
                                                                                                                                                                                target_sharpe=target_sharpe, 
                                                                                                                                                                                to_prot_share=lower_share)

Number of tracks in NA basin: 2692
The principal of the cat bond is: 319894837.909 [USD]
Principal as share of GDP: 0.307


St. Vincent and the Granadines needs a return period of 2500 years <br>
St. Kitts and Nevis needs a return period of 800 years <br>
Cabo verde needs a return period of 200 years <br>
Samoa needs a return period of 300 years <br>
Comoros need a return period of 200 years <br>
St. Lucia needs a return period of <100 years but savings dont really change up onto 2000 years <br>
Dominica needs a return period of 700 years <br>

In [43]:
upper_rp_arr = np.arange(500,2000,500)

bond_metrics_vin = {}
returns_vin = {}
premium_dic_vin = {}
nominal_vin = {}
pay_dam_df_vin = {}
es_metrics_vin = {}
int_grid_vin = {}
imp_per_event_flt_vin = {}
imp_admin_evt_flt_vin = {}

for i in upper_rp_arr:
    bond_metrics, returns, premium_dic, nominal, pay_dam_df, es_metrics, int_grid, imp_per_event_flt, imp_admin_evt_flt = bond_fct.sng_cty_bond(country=52,
                                                                                                                                                prot_share=i, 
                                                                                                                                                rf_rate=rf_rate, 
                                                                                                                                                target_sharpe=target_sharpe, 
                                                                                                                                                to_prot_share=lower_share)
    
    bond_metrics_vin[i] = bond_metrics
    returns_vin[i] = returns
    premium_dic_vin[i] = premium_dic
    nominal_vin[i] = nominal
    pay_dam_df_vin[i] = pay_dam_df
    es_metrics_vin[i] = es_metrics
    int_grid_vin[i] = int_grid
    imp_per_event_flt_vin[i] = imp_per_event_flt
    imp_admin_evt_flt_vin[i] = imp_admin_evt_flt



Number of tracks in NA basin: 1407
The principal of the cat bond is: 1772462278.574 [USD]
Principal as share of GDP: 0.371
Number of tracks in NA basin: 1407
The principal of the cat bond is: 2098898392.031 [USD]
Principal as share of GDP: 0.439
Number of tracks in NA basin: 1407
The principal of the cat bond is: 2153616062.712 [USD]
Principal as share of GDP: 0.451


In [44]:
import simulate_multi_cty_bond as smcb

pay_dam_pool_bs = {308: pay_dam_df_gre}
nominal_pool_bs = {308: nominal_gre}

for i in upper_rp_arr:

    countries_pool = [308, 670]
    pay_dam_pool_it = pay_dam_pool_bs.copy()
    nominal_pool_it = nominal_pool_bs.copy()
    pay_dam_pool_it[670] = pay_dam_df_vin[i]
    nominal_pool_it[670] = nominal_vin[i]
    nom_cty = []
    for cty in nominal_pool_it.keys():
        nom_cty.append(nominal_pool_it[cty])
    nominal_pool = (np.sum(nom_cty))
    exp_loss_ann, att_prob, ann_losses, total_losses, es_metrics, MES_cty = smcb.init_exp_loss_att_prob_simulation(countries_pool, pay_dam_pool_it, nominal_pool, nominal_pool_it, print_prob=True)
    requ_nom = total_losses.max()
    print(requ_nom)


Expected Loss = 0.016884157659914856
Attachment Probability = 0.09382814844453335
0.9204791905159394
Expected Loss = 0.0147564042430903
Attachment Probability = 0.09226101163682437
0.867746100018361
Expected Loss = 0.014443814684202354
Attachment Probability = 0.09226101163682437
0.870671749282183


In [45]:
y = pay_dam_pool_it[670]
z = pay_dam_pool_it[308]
